In [3]:
""" Main Script to evaluate a trained model """
print 



In [3]:
import torch
torch.__version__

'0.4.0a0+200fb22'

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import math
import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 
import prebuilt_attacks as pa

In [5]:
# LOAD THE PRETRAINED (DEFENDED) MODEL (NOTE: THIS HAS TO BE RESNET32)

USE_GPU = torch.cuda.is_available()

VALID_EXPERIMENTS = ['fgsm', 'fullSuite', 'linf', 'linfStadv', 'resnetStadv', 'RTDelta', 
                     'singleFlowDelta']

EXPERIMENT_NAME = 'linfStadv'


classifier_net, cifar_normer = cifar_loader.load_pretrained_cifar_resnet(flavor=32, use_gpu=USE_GPU, 
                                                                    return_normalizer=True) 


defended_state_dict_file = checkpoints.params_to_filename(EXPERIMENT_NAME, 'resnet32')[-1]
print "Loading %s params into resnet32" % defended_state_dict_file 
classifier_net = checkpoints.load_state_dict_from_filename(defended_state_dict_file, classifier_net)

Loading fgsm.resnet32.000099.path.tar params into resnet32
Loading fullSuite.resnet32.000022.path.tar params into resnet32
Loading linf.resnet32.000099.path.tar params into resnet32
Loading linfStadv.resnet32.000156.path.tar params into resnet32
Loading resnetStadv.resnet32.000099.path.tar params into resnet32
Loading RTDelta.resnet32.000099.path.tar params into resnet32
Loading singleFlowDelta.resnet32.000099.path.tar params into resnet32


In [7]:
# LOAD THE EVALUATION DATASET 
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=128, use_gpu=USE_GPU, 
                                         shuffle=False)


Files already downloaded and verified


In [13]:
##############################################################################
#                                                                            #
#             ATTACK BANK                                                    #
#                                                                            #
##############################################################################
ATTACK_LIST = ['delta_fgsm', 'delta_pgd', 'stadv', 'rot_trans', 
               'delta_rot_trans', 'delta_stadv', 'delta_stadv_rot_trans']
def build_attack_ensemble(model, normalizer, attack_list=None,
                          delta_bound=pa.L_INF_BOUND, 
                          flow_bound=pa.FLOW_LINF, 
                          rot_bound=pa.ROT_LINF,
                          trans_bound=pa.TRANS_LINF):
    attack_list = attack_list or ATTACK_LIST
    attack_ensemble = {} 
    
    if 'delta_fgsm' in attack_list:
        attack_ensemble['delta_fgsm'] = pa.build_delta_fgsm(model, 
                                                            normalizer, 
                                                            linf_bound=delta_bound,
                                                            output='eval')
    if 'delta_pgd' in attack_list:
        attack_ensemble['delta_pgd'] = pa.build_delta_pgd(model, normalizer,
                                                          linf_bound=delta_bound,
                                                          output='eval')
    if 'stadv' in attack_list: 
        attack_ensemble['stadv'] = pa.build_stadv_pgd(model, normalizer,
                                                      linf_bound=flow_bound,
                                                      output='eval')
    
    if 'rot_trans' in attack_list:
        attack_ensemble['rot_trans'] = pa.build_rot_trans_pgd(
                                          model, normalizer, 
                                          rot_bound=rot_bound, 
                                          trans_bound=trans_bound,
                                          output='eval')
        
    if 'delta_rot_trans' in attack_list:
        attack_ensemble['delta_rot_trans'] = pa.build_delta_rot_trans_pgd(
                                              model, normalizer, 
                                              delta_bound=delta_bound, 
                                              trans_bound=trans_bound, 
                                              rot_bound=rot_bound,
                                              output='eval')

    if 'delta_stadv' in attack_list:
        attack_ensemble['delta_stadv'] = pa.build_delta_stadv_pgd(
                                            model, normalizer, 
                                            delta_bound=delta_bound, 
                                            flow_bound=flow_bound, 
                                            output='eval')
    
    if 'delta_stadv_rot_trans' in attack_list:
        attack_ensemble['delta_stadv_rot_trans'] =\
            pa.build_delta_stadv_rot_trans_pgd(model, normalizer, 
                                               delta_bound=delta_bound, flow_bound=flow_bound, 
                                               rot_bound=rot_bound, trans_bound=trans_bound, 
                                               output='eval')
    
    return attack_ensemble
    



In [ ]:
def verify_eval(eval_obj, num=4):
    examples, labels = next(iter(val_loader))
    examples = examples.cuda()
    labels = labels.cuda() 
    attack_out = eval_obj.attack_params.attack(examples, labels)
    img_utils.show_images([attack_out[0][:num], examples[:num], 
                           attack_out[0][:num] - examples[:num] + 0.5])

In [24]:
attack_ensemble = build_attack_ensemble(classifier_net, cifar_normer)

eval_obj = adveval.AdversarialEvaluation(classifier_net, cifar_normer, use_gpu=USE_GPU)
eval_output = eval_obj.evaluate_ensemble(val_loader, attack_ensemble, verbose=True, num_minibatches=3)


Starting minibatch 0...
	 (mb: 0) evaluating rot_trans...
	 (mb: 0) evaluating ground...
	 (mb: 0) evaluating delta_fgsm...
	 (mb: 0) evaluating stadv...
	 (mb: 0) evaluating delta_rot_trans...
	 (mb: 0) evaluating delta_stadv_rot_trans...
	 (mb: 0) evaluating delta_stadv...
	 (mb: 0) evaluating delta_pgd...
Starting minibatch 1...
	 (mb: 1) evaluating rot_trans...
	 (mb: 1) evaluating ground...
	 (mb: 1) evaluating delta_fgsm...
	 (mb: 1) evaluating stadv...
	 (mb: 1) evaluating delta_rot_trans...
	 (mb: 1) evaluating delta_stadv_rot_trans...
	 (mb: 1) evaluating delta_stadv...
	 (mb: 1) evaluating delta_pgd...
Starting minibatch 2...
	 (mb: 2) evaluating rot_trans...
	 (mb: 2) evaluating ground...
	 (mb: 2) evaluating delta_fgsm...
	 (mb: 2) evaluating stadv...
	 (mb: 2) evaluating delta_rot_trans...
	 (mb: 2) evaluating delta_stadv_rot_trans...
	 (mb: 2) evaluating delta_stadv...
	 (mb: 2) evaluating delta_pgd...


In [35]:
print "EXPERIMENT: ", EXPERIMENT_NAME
print '-' * 48
print "Attack name", ' ' * (24 - len('attack name')), 'Efficacy (%)'
print '-' * 48
experiment_order = {'ground': 1, 
                    'rot_trans': 2, 
                    'delta_fgsm': 3, 
                    'delta_pgd': 4, 
                    'delta_rot_trans': 5,                    
                    'stadv': 6, 
                    'delta_stadv': 7, 
                    'delta_stadv_rot_trans': 8}

for k, v in sorted(eval_output.items(), key=lambda x: experiment_order[x[0]]):
    pad = 24 - len(k) 
    print k, ' '*pad, '%02.2f' % (v.results['top1'].avg * 100  )
    print '-' * 48

EXPERIMENT:  linfStadv
------------------------------------------------
Attack name               Efficacy (%)
------------------------------------------------
ground                    93.23
------------------------------------------------
rot_trans                 89.58
------------------------------------------------
delta_fgsm                21.35
------------------------------------------------
delta_pgd                 0.00
------------------------------------------------
delta_rot_trans           1.30
------------------------------------------------
stadv                     1.82
------------------------------------------------
delta_stadv               0.00
------------------------------------------------
delta_stadv_rot_trans     0.00
------------------------------------------------
